# Урок 2. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [2]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [3]:
wine_dataset = load_wine()

In [9]:
wine_dataset.data[:, [9, 11, 12]].shape

(178, 3)

In [10]:
# так как мы уже анализировали признаки у этого датасета, взяла 9, 11, 12, считая с нуля
feature_numbers = [9, 11, 12]
x_train, x_test, y_train, y_test = train_test_split(wine_dataset.data[:, feature_numbers], 
                                                    wine_dataset['target'],
                                                    random_state=17)

**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [11]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [14]:
# когда для StratifiedKFold написано использовать random_state, это автоматом означает, 
# что мы устанавливаем shuffle в True ведь? или можно просто не перемешивать, и тогда ставить random_state=None
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [39]:
lr = LogisticRegression(random_state=17, multi_class='ovr')
lr_model = lr.fit(x_train, y_train)
lr_pred = lr.predict(x_test)

In [28]:
cvs = cross_val_score(lr, x_train, y_train, cv=skf, error_score='accuracy')
cvs

array([0.96296296, 0.85185185, 0.92592593, 0.92307692, 0.88461538])

In [29]:
cvs.mean()

0.9096866096866097

In [40]:
accuracy_score(y_test, lr_pred)

0.9111111111111111

### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [31]:
from sklearn.svm import SVC

In [37]:
svc = SVC(random_state=17)
svc_model = svc.fit(x_train, y_train)
svc_pred = svc.predict(x_test)

In [33]:
cvs = cross_val_score(svc, x_train, y_train, cv=skf, error_score='accuracy')
cvs

array([0.7037037 , 0.59259259, 0.7037037 , 0.73076923, 0.73076923])

In [34]:
cvs.mean()

0.6923076923076923

In [38]:
accuracy_score(y_test, svc_pred)

0.6222222222222222

Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [36]:
from sklearn.metrics import classification_report, confusion_matrix

In [42]:
conf_mtrx_lr = confusion_matrix(y_test, lr_pred)
conf_mtrx_lr

array([[ 9,  0,  0],
       [ 0, 19,  0],
       [ 0,  4, 13]], dtype=int64)

Классификатор путает объекты 1-го класса с объектами из второго (то есть он назначил объектам первый класс, а на самом деле он второй) <br>
Если по оси x - предсказанные классы, а по оси y - фактические, то 4 элемента он предсказал как первый класс, когда фактически он второй

In [43]:
conf_mtrx_svc = confusion_matrix(y_test, svc_pred)
conf_mtrx_svc

array([[ 9,  0,  0],
       [ 0, 19,  0],
       [ 2, 15,  0]], dtype=int64)

Классификатор путает объекты нулевого и первого класса с объектами из второго (то есть он им назначил нулевой/первый класс (чаще первый), а на самом деле они из второго класса). Второй класс классификатор вообще не назначил ни разу.

### 5.
Для каждой модели выведите classification report.

In [46]:
print(classification_report(y_test, lr_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.83      1.00      0.90        19
           2       1.00      0.76      0.87        17

    accuracy                           0.91        45
   macro avg       0.94      0.92      0.92        45
weighted avg       0.93      0.91      0.91        45



In [50]:
print(classification_report(y_test, svc_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90         9
           1       0.56      1.00      0.72        19
           2       0.00      0.00      0.00        17

    accuracy                           0.62        45
   macro avg       0.46      0.67      0.54        45
weighted avg       0.40      0.62      0.48        45

